### Inspired by:
* https://www.kaggle.com/sudalairajkumar/a-look-at-different-embeddings
* https://www.kaggle.com/shujian/single-rnn-with-4-folds-v1-9
* http://mlexplained.com/2018/01/13/weight-normalization-and-layer-normalization-explained-normalization-in-deep-learning-part-2/
* https://arxiv.org/abs/1607.06450
* https://github.com/keras-team/keras/issues/3878
* https://www.kaggle.com/lystdo/lstm-with-word2vec-embeddings
* https://www.kaggle.com/jhoward/improved-lstm-baseline-glove-dropout
* https://www.kaggle.com/aquatic/entity-embedding-neural-net
* https://www.kaggle.com/hireme/fun-api-keras-f1-metric-cyclical-learning-rate
* https://ai.google/research/pubs/pub46697
* https://blog.openai.com/quantifying-generalization-in-reinforcement-learning/
* https://www.kaggle.com/rasvob/let-s-try-clr-v3


(and other links in notebook)

Remark:
model overfits like hell...

v6.1:
increased size of conv from 32 -> 100
For commit I have to disable training and tuning stage and fit on whole model, otherwise the running time is longer than 2 hours.

v10:
- use only one model
- add lstm to cnn

v11:
time distributed

v12:
modelcheck+lr on plateu 

v13:
loading best performing

v14:
residual

removing word2vec

v15:
back to base using folds

In [ ]:
import numpy as np # linear algebra
np.set_printoptions(threshold=np.nan)
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
print(os.listdir("../input/embeddings"))
print(os.listdir("../input/embeddings/GoogleNews-vectors-negative300"))

# Any results you write to the current directory are saved as output.

import gensim
from gensim.utils import simple_preprocess
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import Callback
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,f1_score,precision_recall_fscore_support,recall_score,precision_score
from keras import backend as K
from sklearn.utils import class_weight
import matplotlib.pyplot as plt

import tensorflow as tf

# SEED = 2018

# np.random.seed(SEED)
# tf.set_random_seed(SEED)

#https://www.kaggle.com/shujian/single-rnn-with-4-folds-v1-9
def threshold_search(y_true, y_proba):
    best_threshold = 0
    best_score = 0
    for threshold in [i * 0.01 for i in range(100)]:
        score = f1_score(y_true=y_true, y_pred=y_proba > threshold)
        print('\rthreshold = %f | score = %f'%(threshold,score),end='')
        if score > best_score:
            best_threshold = threshold
            best_score = score
    print('\nbest threshold is % f with score %f'%(best_threshold,best_score))
    search_result = {'threshold': best_threshold, 'f1': best_score}
    return search_result

In [ ]:
df = pd.read_csv('../input/train.csv')
df["question_text"].fillna("_##_",inplace=True)
max_len = df['question_text'].apply(lambda x:len(x)).max()
print('max length of sequences:',max_len)
# df = df.sample(frac=0.1)

print('columns:',df.columns)
pd.set_option('display.max_columns',None)
print('df head:',df.head())
print('example of the question text values:',df['question_text'].head().values)
print('what values contains target:',df.target.unique())

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

#dim of vectors
dim = 300
# max words in vocab
num_words = 50000
# max number in questions
max_len = 100 

print('Fiting tokenizer')
## Tokenize the sentences
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(df['question_text'])

print('spliting data')
df_train,df_test = train_test_split(df, random_state=1)

print('text to sequence')
x_train = tokenizer.texts_to_sequences(df_train['question_text'])
x_test = tokenizer.texts_to_sequences(df_test['question_text'])

print('pad sequence')
## Pad the sentences 
x_train = pad_sequences(x_train,maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)

## Get the target values
y_train = df_train['target'].values
y_test = df_test['target'].values

print(x_train.shape)
print(y_train.shape)

In [ ]:
print('Glove ... ')
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open('../input/embeddings/glove.840B.300d/glove.840B.300d.txt'))

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
print(len(all_embs))


word_index = tokenizer.word_index
# num_words = min(num_words, len(word_index))
embedding_matrix_glov = np.random.normal(emb_mean, emb_std, (num_words, dim))
count=0
for word, i in word_index.items():
    if i >= num_words: 
        break
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: 
        embedding_matrix_glov[i] = embedding_vector
    else:
        count += 1
print('embedding matrix size:',embedding_matrix_glov.shape)
print('Number of words not in vocab:',count)

del embeddings_index,all_embs
import gc
gc.collect()

In [ ]:
print('Para...')
EMBEDDING_FILE = '../input/embeddings/paragram_300_sl999/paragram_300_sl999.txt'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding="utf8", errors='ignore') if len(o)>100)

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
print(len(all_embs))


word_index = tokenizer.word_index
# num_words = min(num_words, len(word_index))
embedding_matrix_para = np.random.normal(emb_mean, emb_std, (num_words, dim))
count=0
for word, i in word_index.items():
    if i >= num_words: 
        break
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: 
        embedding_matrix_para[i] = embedding_vector
    else:
        count += 1
print('embedding matrix size:',embedding_matrix_glov.shape)
print('Number of words not in vocab:',count)

del embeddings_index,all_embs
import gc
gc.collect()

In [ ]:
print('Wiki...')
EMBEDDING_FILE = '../input/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if len(o)>100)

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()

print(len(all_embs))

word_index = tokenizer.word_index
embedding_matrix_wiki = np.random.normal(emb_mean, emb_std, (num_words, dim))

count=0
for word, i in word_index.items():
    if i >= num_words: 
        break
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: 
        embedding_matrix_wiki[i] = embedding_vector
    else:
        count += 1
print('embedding matrix size:',embedding_matrix_wiki.shape)
print('Number of words not in vocab:',count)

del embeddings_index,all_embs
import gc
gc.collect()

In [ ]:
matrixes = [embedding_matrix_glov,embedding_matrix_wiki,embedding_matrix_para]

matrix = np.mean(matrixes,axis=0)

del embedding_matrix_glov,embedding_matrix_wiki,embedding_matrix_para
import gc
gc.collect()

In [ ]:
from keras.layers import Dense, Input,Embedding, Dropout, Activation, CuDNNLSTM,BatchNormalization,concatenate,SpatialDropout1D
from keras.layers import Bidirectional, GlobalMaxPool1D, Concatenate, GlobalAveragePooling1D,Average,Conv1D,GlobalMaxPooling1D,AlphaDropout
from keras.layers import MaxPooling1D,UpSampling1D,RepeatVector,LSTM,TimeDistributed,Flatten, CuDNNGRU, Add
from keras.models import Model
from keras.callbacks import Callback,EarlyStopping,ModelCheckpoint, ReduceLROnPlateau
from keras.engine import Layer
from keras.initializers import Ones, Zeros
import keras.backend as K
from keras import regularizers
from keras import constraints
from keras import optimizers
from keras import initializers
from sklearn.model_selection import KFold, StratifiedKFold

def f1(y_true, y_pred):
    '''
    metric from here 
    https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras
    '''
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))



#overfitting:
# https://stackoverflow.com/questions/43156397/how-to-avoid-overfitting-in-the-given-convnet
# http://www.wildml.com/2015/12/implementing-a-cnn-for-text-classification-in-tensorflow/
# https://blog.openai.com/quantifying-generalization-in-reinforcement-learning/
#word2vec   
def get_model(adam):
    inp1 = Input(shape=(max_len,))
    x = Embedding(num_words, dim, weights=[matrix],trainable = False,)(inp1)
    avg_pool = GlobalAveragePooling1D()(x)

    #classification dense net
    x = Dense(max_len, activation="relu")(avg_pool)
    x = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=inp1, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy',f1])
    print(model.summary())
    return model

def get_opt():
    # https://ai.google/research/pubs/pub46697
    adam = optimizers.Adam()
    print('LR:',K.eval(adam.lr))
    return adam

def get_batch_size(opt):
    # 0.001 = learning rate in adam
    # optimal batch size ~ eps *N, where eps = learning rate and N = training size
    batch_size = int(x_train.shape[0]*K.eval(opt.lr))
    print('Batch size = ',batch_size)
    return batch_size

patience = 2

best_model=None
all_results = {}

splits = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=14).split(x_train, y_train))
for idx, (train_idx, valid_idx) in enumerate(splits):
    print('----'+str(idx)+'-----')
    X_train1 = x_train[train_idx]
    y_train1 = y_train[train_idx]
    X_val = x_train[valid_idx]
    y_val = y_train[valid_idx]
    model_file = 'model_'+str(idx)+'.h5'
    modelcheck = ModelCheckpoint(model_file,save_best_only=True)
    stop = EarlyStopping(patience=patience)
    reduce = ReduceLROnPlateau(patience=1,verbose=2)
    
    adam = get_opt()
    batch_size = get_batch_size(adam)
    
    model = get_model(adam)
    
    history = model.fit(X_train1,y_train1, 
                      batch_size=batch_size, 
                      validation_data=(X_val,y_val),
                      epochs=100,
                      #overfits rather soon
                      callbacks=[modelcheck,stop,reduce],
                      verbose=2)

    print('training finished...')

    #load best performing

    model.load_weights(model_file)

    #for train set
    y_pred = model.predict(X_train1,batch_size=batch_size, verbose=1)
    search_result = threshold_search(y_train1, y_pred)
    print(search_result)
    y_pred = y_pred>search_result['threshold']
    y_pred = y_pred.astype(int)

    print('RESULTS ON TRAINING SET:\n',classification_report(y_train1,y_pred))

    #for test set
    y_pred = model.predict(x_test,batch_size=batch_size, verbose=1)
    search_result = threshold_search(y_test, y_pred)
    print(search_result)
    y_pred = y_pred>search_result['threshold']
    y_pred = y_pred.astype(int)

    print('RESULTS ON TEST SET:\n',classification_report(y_test,y_pred))

    all_results[model_file] = search_result['f1']    
    
    if best_model is None or best_model['f1']  < search_result['f1']:
        best_model={'model':model_file,'f1':search_result['f1']}
        
print('-'*80)
print(all_results)
print('-'*80)
print(best_model)


In [ ]:
adam = get_opt()
batch_size = get_batch_size(adam)

model = get_model(adam)
model.load_weights(best_model['model'])
#for test set
y_pred = model.predict(x_test,batch_size=batch_size, verbose=1)
search_result = threshold_search(y_test, y_pred)
print(search_result)
y_pred = y_pred>search_result['threshold']
y_pred = y_pred.astype(int)
print('RESULTS ON TEST SET:\n',classification_report(y_test,y_pred))

In [ ]:
#submission
print('Loading test data...')
df_final = pd.read_csv('../input/test.csv')
df_final["question_text"].fillna("_##_", inplace=True)

x_final=tokenizer.texts_to_sequences(df_final['question_text'])
x_final = pad_sequences(x_final,maxlen=max_len)

y_pred = model.predict(x_final,batch_size=batch_size,verbose=1)
y_pred = y_pred > search_result['threshold']
y_pred = y_pred.astype(int)
print(y_pred[:5])

df_subm = pd.DataFrame()
df_subm['qid'] = df_final.qid
df_subm['prediction'] = y_pred
print(df_subm.head())
df_subm.to_csv('submission.csv', index=False)